In [7]:

import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt
import pandas_datareader as web
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib.dates as dates
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [9]:
data=pd.read_csv('task1/AMZNtest (1).csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'AMZNtest(1).csv'

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data['Date'],y=data['Open'].rolling(window=28).mean(),name="netflix"),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="AMAZON")
fig.update_xaxes(title_text="year")
fig.update_yaxes(title_text="prices",secondary_y=False)
fig.show()

In [ ]:
t_data=pd.read_csv("/content/sample_data/AMZNtest (1).csv")
t_data.head()

In [ ]:
print(len(data))
print(len(t_data))

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Open'].values.reshape(-1,1))

In [ ]:
prediction_days = 40

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x - prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
def LSTM_model():

    model = Sequential()
    model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1],1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    return model
model = LSTM_model()
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
checkpointer = ModelCheckpoint(filepath = 'weights_best.hdf5', verbose = 1, save_best_only = True)
model.fit(x_train, y_train, epochs=20, batch_size=32,callbacks=[checkpointer])

In [ ]:
actual_prices = t_data['Open'].values
total_dataset = pd.concat((data['Open'], t_data['Open']), axis=0)

model_inputs = total_dataset[len(total_dataset)-len(t_data)-prediction_days:].values
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.transform(model_inputs)
x_test = []
for x in range(prediction_days,len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x,0])

x_test = np.array(x_test)
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

predicted_prices = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)
x=[]
values=[]
for i in range(5):
    x+=[i*5]
    values+=[t_data['Date'].values[i*5]]
print(x)
print(values)

In [ ]:
plt.xticks(x,values)
plt.plot(actual_prices, color='black', label=f"actual price")
plt.plot(predicted_prices, color='green', label=f"predicted price")
plt.title(f"AMAZON")
plt.xlabel("Date")
plt.ylabel(f"price")
plt.legend()
plt.show()

In [ ]:
real_data = [model_inputs[len(model_inputs)+1-prediction_days:len(model_inputs+1),0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data,(real_data.shape[0],real_data.shape[1],1))
print(real_data.shape)
prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f"prediction: {int(prediction[0][0])}")